<a href="https://colab.research.google.com/github/Droslj/scATAC-seq-complete-/blob/Google-colab/scATAC_seq_(1)_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scATAC seq, based on scATAC seq processing Galaxy tutorials (scATAC preprocessing (2), Standard scATAC seq processing pipeline (1) )
AD Objects created in Galaxy using customized Galaxy WF with Snapatac2 and imported
(1) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-preprocessing-tenx/tutorial.html#mapping-reads-to-a-reference-genome, (2) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-standard-processing-snapatac2/tutorial.html
Data taken from the following NCBI study:
Metabolic adaptation pilots the differentiation of human hematopoietic cells (https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1015713)
Import Anndata objects for two biological replicates, SRR26046013 (cells treated with AOA inhibitor) and SRR26046019 (untreated cells)
Perform Differential accesibility analysis

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab

In [ ]:
condacolab.install()

In [ ]:
!conda --version

In [ ]:
!which conda

In [ ]:
!conda config --add channels conda-forge

In [ ]:
!conda config --add channels bioconda

In [ ]:
!pip install snapatac2 -q

In [ ]:
!pip show snapatac2

In [ ]:
import snapatac2 as snap

In [ ]:
!pip install umap-learn



In [ ]:
import umap.umap_ as umap


In [ ]:
from umap import UMAP

In [ ]:
!pip install scanpy -q

In [ ]:
import scanpy as sc

In [ ]:
pip show scanpy

In [ ]:
!pip install magic-impute -q

In [ ]:
!pip install leidenalg -q

In [ ]:
import numpy as np

In [ ]:
import anndata as ad

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go

In [ ]:
import numpy as np

In [ ]:
from scipy import stats

# Import reads from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
adata_concat = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/adata_concat.h5ad')

In [ ]:
adata_concat

# Differential accesibility analysis

In [ ]:
# Assuming adata is your AnnData object
condition1 = adata_concat[adata_concat.obs['Treatment'] == 'Treated w/AOA']
condition2 = adata_concat[adata_concat.obs['Treatment'] == 'Untreated']

In [ ]:
#Original code
#pvals = []
# for peak in adata_concat.var_names: _, pval = stats.ttest_ind(condition1[:, peak].X.toarray(), condition2[:, peak].X.toarray())
# pvals.append(pval)

In [ ]:
##Improved code with progress tracking
#import tqdm

## Perform a t-test for each peak/gene
#pvals = []
#for peak in tqdm.tqdm(adata_concat.var_names, desc="Calculating p-values"):
#    _, pval = stats.ttest_ind(condition1[:, peak].X.toarray(), condition2[:, peak].X.toarray())
#    pvals.append(pval)

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
import tqdm

num_cores = multiprocessing.cpu_count()

pvals = Parallel(n_jobs=num_cores)(
       delayed(stats.ttest_ind)(condition1[:, peak].X.toarray(), condition2[:, peak].X.toarray())
       for peak in tqdm.tqdm(adata_concat.var_names, desc="Calculating p-values")
   )
pvals = [p[1] for p in pvals]  # Extract p-values from results

In [ ]:
# Adjust p-values for multiple testing (e.g., using Benjamini-Hochberg)
from statsmodels.stats.multitest import multipletests
_, pvals_adj, _, _ = multipletests(pvals, method='fdr_bh')

In [ ]:
# Add p-values to AnnData object
adata_concat.var['pvals'] = pvals
adata_concat.var['pvals_adj'] = pvals_adj

In [ ]:
Filter Significant DARs: Filter the results to retain only those regions that are significantly differentially accessible after multiple testing correction.

In [ ]:
significant_dars = adata.var[adata.var['pvals_adj'] < 0.05]

In [ ]:
Visualize Results: Visualize the differentially accessible regions using various plotting functions available in scanpy or other visualization libraries.

In [ ]:
sc.pl.heatmap(adata, var_names=significant_dars.index, groupby='condition')

Interpret Results: Interpret the biological significance of the differentially accessible regions. This might involve looking at the genes associated with these regions and understanding their roles in the biological conditions you are studying.